In [ ]:
import os
os.environ["GOOGLE_API_KEY"] = "use your own"

In [ ]:
import os
import base64
from google import genai
from google.genai import types
from google.genai.types import GenerateContentConfig

def initialize_client():
    api_key = os.environ.get("GOOGLE_API_KEY")
    if not api_key:
        raise ValueError("Missing API key. Set GOOGLE_API_KEY in environment variables.")
    return genai.Client(api_key=api_key)

chat_history = []

def generate_chat_response(client, user_input):
    global chat_history
    chat_history.append(types.Content(role="user", parts=[types.Part.from_text(text=user_input)]))

    config = GenerateContentConfig(
        temperature=1,
        top_p=0.95,
        top_k=64,
        max_output_tokens=8192,
        response_mime_type="text/plain",
    )

    model = "gemini-2.0-pro-exp-02-05"

    response = client.models.generate_content(
        model=model,
        contents=chat_history,
        config=config,
    )

    bot_reply = response.text
    chat_history.append(types.Content(role="model", parts=[types.Part.from_text(text=bot_reply)]))

    return bot_reply

def extract_text_from_image(client, image_path):
    with open(image_path, "rb") as image_file:
        encoded_image = base64.b64encode(image_file.read()).decode("utf-8")

    response = client.models.generate_content(
        model="gemini-2.0-pro-exp-02-05",
        contents=[
            types.Content(
                role="user",
                parts=[
                    types.Part.from_text("Extract text from this image"),
                    types.Part.from_data(mime_type="image/jpeg", data=encoded_image),
                ],
            )
        ],
    )

    return response.text

def main():
    client = initialize_client()
    print("Welcome! This chatbot allows you to chat with AI and extract text from images. Simply type your message, or provide an image path to extract text.")

    while True:
        user_input = input("You: ")

        if user_input.lower() == "exit":
            print("Goodbye!")
            break
        elif os.path.exists(user_input):
            extracted_text = extract_text_from_image(client, user_input)
            print("Extracted Text:", extracted_text)
        else:
            response = generate_chat_response(client, user_input)
            print("AI:", response)

if __name__ == "__main__":
    main()

Welcome! This chatbot allows you to chat with AI and extract text from images. Simply type your message, or provide an image path to extract text.
You: hey
AI: Hey there! How can I help you today?

You: what is python
AI: Python is a high-level, general-purpose, interpreted programming language.  Here's a breakdown of what that means, and what makes Python so popular:

**Key Characteristics and Concepts:**

*   **High-Level:**  This means Python abstracts away a lot of the low-level details of computer hardware (like memory management).  You write code that's closer to human language than to machine code, making it easier to learn and use.  You don't have to worry about things like manually allocating memory or managing pointers (as you would in languages like C or C++).

*   **General-Purpose:** Python can be used for a very wide variety of tasks, including:
    *   **Web Development:**  Frameworks like Django and Flask make it easy to build web applications.
    *   **Data Science & 

In [1]:
import os
os.environ["GEMINI_API_KEY"] = "use your own"

In [3]:
import base64
import os
import json
from google import genai
from google.genai import types

def save_binary_file(file_name, data):
    with open(file_name, "wb") as f:
        f.write(data)

def save_chat_history(chat_history, file_name="chat_history.json"):
    with open(file_name, "w") as f:
        json.dump([{"role": content.role, "text": content.parts[0].text} for content in chat_history], f, indent=4)

def initialize_client():
    api_key = os.environ.get("GEMINI_API_KEY")
    if not api_key:
        raise ValueError("Missing API key. Set GEMINI_API_KEY in environment variables.")
    return genai.Client(api_key=api_key)

def generate():
    client = initialize_client()
    model = "gemini-2.0-flash-exp-image-generation"
    chat_history = []

    print("Welcome! You can generate images and text responses. Type your prompt below.")

    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            print("Goodbye! Saving chat history...")
            save_chat_history(chat_history)
            break

        chat_history.append(types.Content(role="user", parts=[types.Part.from_text(text=user_input)]))

        generate_content_config = types.GenerateContentConfig(
            temperature=1,
            top_p=0.95,
            top_k=40,
            max_output_tokens=8192,
            response_modalities=["image", "text"],
            response_mime_type="text/plain",
        )

        for chunk in client.models.generate_content_stream(
            model=model,
            contents=chat_history,
            config=generate_content_config,
        ):
            if chunk.candidates and chunk.candidates[0].content and chunk.candidates[0].content.parts:
                part = chunk.candidates[0].content.parts[0]
                if part.inline_data:
                    file_name = "generated_image.png"
                    save_binary_file(file_name, part.inline_data.data)
                    print(f"Image saved as: {file_name}")
                else:
                    response_text = chunk.text
                    print("AI:", response_text)
                    chat_history.append(types.Content(role="model", parts=[types.Part.from_text(text=response_text)]))

if __name__ == "__main__":
    generate()

Welcome! You can generate images and text responses. Type your prompt below.
You: flower
Image saved as: generated_image.png
You: zebra
Image saved as: generated_image.png
You: cow
Image saved as: generated_image.png
You: a cow eating grass
Image saved as: generated_image.png
You: a cow eating a beautiful lady
AI: I'm unable to create an image based on this request. Is
AI:  there something else I can help you visualize?
You: dead body
AI: 
I'm unable to create images that depict violence or are graphic in nature. Is there something else I can help you create?
You: blood  sheed
AI: I
AI: 'm unable to create images that depict violence or are graphic in nature. Is
AI:  there something else I can help you create?
You: a gorgeous beautiful leady named faria
Image saved as: generated_image.png
You: exit
Goodbye! Saving chat history...
